In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# 1. Загрузка данных
raw_data = pd.read_csv('/content/drive/MyDrive/HAKATON/COX-2.csv', sep=';')

In [ ]:
raw_data.head(5)

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,Action Type,Standard Text Value,Value
0,CHEMBL70104,NaN,NaN,364.46,0.0,3.49,3,CC1(C)OC(=O)C(OC2CCCCC2)=C1c1ccc(S(C)(=O)=O)cc1,IC50,'=',...,CHEMBL1131821,1,Scientific Literature,Bioorg Med Chem Lett,1999.0,NaN,NaN,NaN,NaN,0.04
1,CHEMBL96098,NaN,NaN,372.88,0.0,4.43,34,CCc1ccc(-c2ncc(Cl)cc2-c2ccc(S(C)(=O)=O)cc2)cn1,IC50,'=',...,CHEMBL1130609,1,Scientific Literature,Bioorg Med Chem Lett,1998.0,NaN,NaN,NaN,NaN,1.70
2,CHEMBL97179,NaN,NaN,413.90,1.0,5.19,5,CCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(OC)...,IC50,'=',...,CHEMBL1133457,1,Scientific Literature,J Med Chem,2000.0,NaN,NaN,NaN,NaN,0.05
3,CHEMBL6,INDOMETHACIN,4.0,357.79,0.0,3.93,Indomethacin,COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c1ccc(Cl)cc1,IC50,'<',...,CHEMBL1131754,1,Scientific Literature,Bioorg Med Chem Lett,1999.0,NaN,NaN,NaN,NaN,200.00
4,CHEMBL355781,NaN,NaN,390.46,0.0,4.25,21,CS(=O)(=O)c1ccc(-c2csc(CC(=O)O)c2-c2ccc(F)cc2)cc1,IC50,'>',...,CHEMBL1128908,1,Scientific Literature,Bioorg Med Chem Lett,1996.0,CHEMBL3308860,NaN,NaN,NaN,10.00


In [ ]:
print(len(raw_data))

7979


In [ ]:
raw_data.dtypes

,0
Molecule ChEMBL ID,object
Molecule Name,object
Molecule Max Phase,float64
Molecular Weight,float64
#RO5 Violations,float64
AlogP,float64
Compound Key,object
Smiles,object
Standard Type,object
Standard Relation,object


In [ ]:
raw_data['Standard Value']

,Standard Value
0,40.0
1,1700.0
2,50.0
3,200.0
4,10000.0
...,...
7974,40.0
7975,370000.0
7976,1100.0
7977,880.0


In [ ]:
# 2. Удаление нерелевантных записей (записей, у которых отсутствует значение Standard Value)
data_act = raw_data[raw_data["Standard Value"].notna()]

In [ ]:
set(data_act['Standard Relation'])

{"'<'", "'='", "'>'", "'>='"}

In [ ]:
# Удаление строк с неизвестным точным значением целевого свойства (<, >, >= вместо =)
data_act = data_act[data_act['Standard Relation'] == "'='"]

In [ ]:
len(data_act)

5948

In [ ]:
# 3. Привожу записи к единому формату
# Вывожу все типы данных для Standard Units и отделяю по ним, формируя отдельные датасеты
set(data_act['Standard Units'])

{'%', 'nM', 'ug', 'ug.mL-1'}

In [ ]:
data_act_pr = data_act[data_act['Standard Units'] == '%']
len(data_act_pr)

1

In [ ]:
data_act_nm = data_act[data_act['Standard Units'] == 'nM']
len(data_act_nm)

5927

In [ ]:
data_act_ug = data_act[data_act['Standard Units'] == 'ug']
len(data_act_ug)

1

In [ ]:
data_act_ug_ml = data_act[data_act['Standard Units'] == 'ug.mL-1']
len(data_act_ug_ml)

19

In [ ]:
data_act_ug_ml['Standard Value']

,Standard Value
906,17.70
1733,54.42
2149,11.35
3668,25.02
5578,13.67
5909,19.95
5939,24.59
6039,32.93
6130,23.74
6178,12.82


In [ ]:
# Так как типы данных % и ug присутствуют всего в штучном экземпляре в датасете, а также не очень понятно, как из этих типов данных переводить, было решено не включать их в итоговый датасет
# Переведу мкг/мл в нмоль/л . моль/л = (г/л) / (моль/г). мкг/мл = мг/л = 10 ^ (-3) г/л. моль/л = 10 ^ 9 нмоль/л, в итоге получаем нмоль/л = 10 ^ 6 * (мкг/мл) / (моль/г), что и пересчитано ниже
data_act_ug_ml['Standard Value'] = data_act_ug_ml['Standard Value'] * 10 ** 6 / data_act_ug_ml['Molecular Weight']

/tmp/ipython-input-18-1619602230.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act_ug_ml['Standard Value'] = data_act_ug_ml['Standard Value'] * 10 ** 6 / data_act_ug_ml['Molecular Weight']


In [ ]:
data_act_ug_ml['Standard Value']

,Standard Value
906,3.484389e+04
1733,1.153088e+05
2149,2.234340e+04
3668,5.541160e+04
5578,2.896493e+04
5909,4.560000e+04
5939,5.192803e+04
6039,7.261301e+04
6130,5.426286e+04
6178,2.707269e+04


In [ ]:
# Так как значения я изменил, единицы измерения тоже меняю, чтобы не было путаницы
data_act_ug_ml['Standard Units'] = 'nM'

/tmp/ipython-input-20-930595832.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_act_ug_ml['Standard Units'] = 'nM'


In [ ]:
# Соединяю датасеты с едицами измерения нМ и бывшиими мкг/мл в один датасет
data_act_ost = pd.concat([data_act_nm, data_act_ug_ml], axis=0)
len(data_act_ost)

5946

In [ ]:
# 4. Удаление дубликатов
data_act_ost.drop_duplicates()
len(data_act_ost)

5946

In [ ]:
data_act_ost.head(5)

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,Action Type,Standard Text Value,Value
0,CHEMBL70104,NaN,NaN,364.46,0.0,3.49,3,CC1(C)OC(=O)C(OC2CCCCC2)=C1c1ccc(S(C)(=O)=O)cc1,IC50,'=',...,CHEMBL1131821,1,Scientific Literature,Bioorg Med Chem Lett,1999.0,NaN,NaN,NaN,NaN,0.04
1,CHEMBL96098,NaN,NaN,372.88,0.0,4.43,34,CCc1ccc(-c2ncc(Cl)cc2-c2ccc(S(C)(=O)=O)cc2)cn1,IC50,'=',...,CHEMBL1130609,1,Scientific Literature,Bioorg Med Chem Lett,1998.0,NaN,NaN,NaN,NaN,1.70
2,CHEMBL97179,NaN,NaN,413.90,1.0,5.19,5,CCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(OC)...,IC50,'=',...,CHEMBL1133457,1,Scientific Literature,J Med Chem,2000.0,NaN,NaN,NaN,NaN,0.05
5,CHEMBL276436,NaN,NaN,316.49,1.0,5.66,32,CC(C)CCCC(=O)c1cc(C(C)(C)C)c2c(c1)C(C)(C)CO2,IC50,'=',...,CHEMBL1131314,1,Scientific Literature,J Med Chem,1998.0,NaN,NaN,NaN,NaN,4.50
6,CHEMBL95380,NaN,NaN,574.08,2.0,5.28,36,COc1ccc2c(c1)c(CC(=O)NCCOC(=O)/C=C/c1ccc(N(C)C...,IC50,'=',...,CHEMBL1133457,1,Scientific Literature,J Med Chem,2000.0,NaN,NaN,NaN,NaN,0.19


In [ ]:
# Удаление молекул с отсутствующим smiles. Также я здесь удаляю столбцы, в которых есть пустые значения и строки с пустыми значениями
data_clean = data_act_ost.dropna(subset=["Smiles"])
data_clean = data_clean.dropna(axis=1)
data_clean = data_clean.dropna()
len(data_clean)

5945

In [ ]:
data_clean.head(5)

,Molecule ChEMBL ID,Molecular Weight,Compound Key,Smiles,Standard Type,Standard Relation,Standard Value,Standard Units,Uo Units,Potential Duplicate,...,BAO Format ID,BAO Label,Target ChEMBL ID,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Value
0,CHEMBL70104,364.46,3,CC1(C)OC(=O)C(OC2CCCCC2)=C1c1ccc(S(C)(=O)=O)cc1,IC50,'=',40.0,nM,UO_0000065,0,...,BAO_0000221,tissue-based format,CHEMBL230,Cyclooxygenase-2,Homo sapiens,SINGLE PROTEIN,CHEMBL1131821,1,Scientific Literature,0.04
1,CHEMBL96098,372.88,34,CCc1ccc(-c2ncc(Cl)cc2-c2ccc(S(C)(=O)=O)cc2)cn1,IC50,'=',1700.0,nM,UO_0000065,0,...,BAO_0000357,single protein format,CHEMBL230,Cyclooxygenase-2,Homo sapiens,SINGLE PROTEIN,CHEMBL1130609,1,Scientific Literature,1.70
2,CHEMBL97179,413.90,5,CCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(OC)...,IC50,'=',50.0,nM,UO_0000065,0,...,BAO_0000357,single protein format,CHEMBL230,Cyclooxygenase-2,Homo sapiens,SINGLE PROTEIN,CHEMBL1133457,1,Scientific Literature,0.05
5,CHEMBL276436,316.49,32,CC(C)CCCC(=O)c1cc(C(C)(C)C)c2c(c1)C(C)(C)CO2,IC50,'=',4500.0,nM,UO_0000065,0,...,BAO_0000019,assay format,CHEMBL230,Cyclooxygenase-2,Homo sapiens,SINGLE PROTEIN,CHEMBL1131314,1,Scientific Literature,4.50
6,CHEMBL95380,574.08,36,COc1ccc2c(c1)c(CC(=O)NCCOC(=O)/C=C/c1ccc(N(C)C...,IC50,'=',190.0,nM,UO_0000065,0,...,BAO_0000357,single protein format,CHEMBL230,Cyclooxygenase-2,Homo sapiens,SINGLE PROTEIN,CHEMBL1133457,1,Scientific Literature,0.19


In [ ]:
data_clean.dtypes

,0
Molecule ChEMBL ID,object
Molecular Weight,float64
Compound Key,object
Smiles,object
Standard Type,object
Standard Relation,object
Standard Value,float64
Standard Units,object
Uo Units,object
Potential Duplicate,int64


In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 48.4 MB/s eta 0:00:00


In [ ]:
import rdkit

In [ ]:
# 5. Проверка корректности молекул. Пробую по каждому smiles собрать молекулу. rdkit в случае некорректного smiles пишет в консоль. Вывод чист, значит все молекулы корректны
for molecule in data_clean['Smiles']:
  try:
    mol = rdkit.Chem.rdmolfiles.MolFromSmiles(molecule)
    if mol is None:
      print(molecule)
  except Exception:
    print(molecule)

In [ ]:
data_clean.to_csv("/content/drive/MyDrive/HAKATON/clean_data.csv", index=False)